In [1]:
import pandas as pd
import requests

import glob

#CONEXAO COM GSHEETS
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from google.oauth2 import service_account
import gspread_dataframe as gd

import json
import base64

#Puxar os Secrets Cadastrados no Fury
from melitk import melipass    
from google.cloud import bigquery

pd.options.display.float_format = '{:,.2f}'.format


In [2]:
def conecta_gsheets():
    #GoogleSheets
    scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
    #Estabelecendo a conexão
    creds = ServiceAccountCredentials.from_json_keyfile_dict(credSheets,scope)
    client = gspread.authorize(creds)
    return(client)


In [3]:
#Conexão Sheets
credSheets = json.loads(base64.b64decode(melipass.get_secret('GOOGLE_CRED')))

#Conexão BQ
key_info = melipass.get_secret('BQ_CHAVE')
json_acct_info = json.loads(base64.b64decode(key_info))
credentials = service_account.Credentials.from_service_account_info(
    json_acct_info)
scoped_credentials = credentials.with_scopes(
    ['https://www.googleapis.com/auth/cloud-platform'])
bq_client = bigquery.Client(project='meli-bi-data', credentials=scoped_credentials)

In [4]:
def monta_df(res):
    col_df = []
    for c in res.result().schema:
        col_df.append(c.name) 

    lista_dados = []
    for row in res.result():
        lista_dados.append(list(row))
    return pd.DataFrame(lista_dados,columns=col_df)

In [5]:
def conecta_gsheets():
    #GoogleSheets
    scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
    #Estabelecendo a conexão
    creds = ServiceAccountCredentials.from_json_keyfile_dict(credSheets,scope)
    client = gspread.authorize(creds)
    return(client)

In [6]:
client = conecta_gsheets()
sheet = client.open_by_key('15uzNPzAYb-qONOsDs0Hwe7LwwP23dVcLgBje_m0s5_o')
ws = sheet.worksheet('TGMV')

df_parametros= pd.DataFrame(ws.get_all_values())
headers = df_parametros.iloc[5]
df_parametros = pd.DataFrame(df_parametros.values[6:], columns=headers)

In [7]:
query = """
DECLARE DOM2 STRING;
SET DOM2 = '{}';

SELECT
  DOM2,  
  CUS_CUST_ID_SEL,
  round(sum(F_TGMV_LC),2) as TOTALGMV,
  sum(case when DOM_AGG_2 = DOM2 then round(F_TGMV_LC,2) ELSE 0 END) as DOMGMV,
  sum(case when LOGISTIC_TYPE_ORDER in ('FBM','FLEX','XD') and DOM_AGG_2 = DOM2 then round(F_TGMV_LC,2) ELSE 0 END) as ENVGMV

FROM `meli-bi-data.EXPLOTACION.DATAMART_MKPLC` 

WHERE 1=1 
  and DIA_SERVER >= current_date()-30
  and SITE_ID = 'MLB'
group by 1,2
having DOMGMV > 0"""


In [8]:
df_parametros.head()

5,02. Items Domain Agg 1,02. Items Domain Agg 2,01. Orders TGMV LC + Forecast,01. Orders TGMV LC + Forecast,01. Orders TGMV LC + Forecast,01. Orders TGMV LC + Forecast,01. Orders TGMV LC + Forecast,01. Orders TGMV LC + Forecast,INDEX > 1.1,ESTRATÉGICO,,,
0,BOOKS,MULTIMEDIA & OTHER E!,"4.215,57","1.448.786,96","1.074.092,24",1,"343,68","254,79",S,N,,,
1,BOOKS,MULTIMEDIA & OTHER E!,"81.361,46","23.906.961,79","19.883.773,62",1,"293,84","244,39",S,N,,,
2,BOOKS,MULTIMEDIA & OTHER E!,"29.025,38","5.149.455,55","4.786.615,08",1,"177,41","164,91",S,N,,,
3,HOUSEHOLD ITEMS,FIREPLACES,"675.560,55","944.970,00","2.595.753,97",1,"1,4","3,84",S,S,,,
4,FOOTWEAR,BOOTS,"24.392.524,99","32.417.230,07","62.268.150,78",1,"1,33","2,55",S,S,,,


In [9]:
dom2 = df_parametros[df_parametros['ESTRATÉGICO']=='S']['02. Items Domain Agg 2'].to_list()

In [11]:
resumo = {}
base = {}

In [15]:
for d in dom2:
    print('Apurando dados para {}'.format(d))
    res =  bq_client.query(query.format(d))
    df = monta_df(res)
    df['DOM%'] = df['DOMGMV']/df['TOTALGMV']
    df_res = df['DOM%'].apply(float).describe().to_frame().reset_index().pivot_table(columns='index',values='DOM%')
    df_res['DOM']=d
    df['DOM']=d
    resumo[d]=df_res
    base[d]=df

Apurando dados para FIREPLACES
Apurando dados para BOOTS
Apurando dados para CLOTHING
Apurando dados para BEDROOM TEXTILES
Apurando dados para HATS
Apurando dados para BASICS
Apurando dados para BELT
Apurando dados para MULTIMEDIA
Apurando dados para SNEAKERS
Apurando dados para FLATS
Apurando dados para CARPETS
Apurando dados para SCHEDULED SERVICES
Apurando dados para FAUCETS
Apurando dados para SPORTS ACCESSORIES
Apurando dados para FRAGRANCES
Apurando dados para TEXTIL INDUSTRY
Apurando dados para WATCH
Apurando dados para SEX TOYS
Apurando dados para PET ACCESSORIES
Apurando dados para SMALL HOUSE APLLIANCES
Apurando dados para KIDS APPAREL
Apurando dados para KITCHEN
Apurando dados para EROTIC PARTY SUPPLY & COSTUMES
Apurando dados para HEALTH ELECTRONICS
Apurando dados para OTHERS BABY
Apurando dados para LENSES, PARTS & OTHERS
Apurando dados para PERCUSION INSTRUMENTS
Apurando dados para MUSICAL KEYBOARDS
Apurando dados para DJ CONSOLES & MIXERS
Apurando dados para PARTY DECORA

In [17]:
res_penetracao = pd.concat(resumo.values())

In [18]:
base_completa = pd.concat(base.values())

In [19]:
base_completa.head()

,DOM2,CUS_CUST_ID_SEL,TOTALGMV,DOMGMV,ENVGMV,DOM%,DOM
0,FIREPLACES,"219,324,699.00",19221897.22,7567.57,50.75,0.0003936952691707296518361052812,FIREPLACES
1,FIREPLACES,"209,063,821.00",85254.05,31937.9,0,0.3746203259551892256145015985,FIREPLACES
2,FIREPLACES,"136,610,540.00",29648.62,29086.11,0,0.9810274474832218160575433190,FIREPLACES
3,FIREPLACES,"175,357,767.00",1234.15,552.93,0,0.4480249564477575659360693595,FIREPLACES
4,FIREPLACES,"362,659,034.00",118593.03,7981.65,391.92,0.06730285919838627953093027474,FIREPLACES


In [30]:
base_estudo = base_completa.groupby(by='CUS_CUST_ID_SEL')[['TOTALGMV','DOMGMV','ENVGMV','DOM']].agg({'TOTALGMV':'mean','DOMGMV':'sum','DOM':'count','ENVGMV':'sum'}).reset_index()

In [40]:
base_estudo.head()

,CUS_CUST_ID_SEL,TOTALGMV,DOMGMV,DOM,ENVGMV
0,"2,126.00",62.67,62.67,1,0
1,"4,651.00","1,449.35",1449.35,1,0
2,"5,824.00",276.99,276.99,1,0
3,"14,667.00","1,553.02",995.06,2,0
4,"20,351.00","17,334.08",8096.77,3,8096.77


In [46]:
def elegibilidade(reg):
    if float(reg['DOMGMV'])/float(reg['TOTALGMV'])<0.75:
        return 'Não Opera GMV Suficiente nos Dominios Chaves'
    elif float(reg['ENVGMV'])==0:
        return 'Não Opera nos Modais'
    return 'Aprovado'
    

In [47]:
base_estudo.dtypes

CUS_CUST_ID_SEL    float64
TOTALGMV           float64
DOMGMV              object
DOM                  int64
ENVGMV              object
dtype: object

In [49]:
base_estudo['status'] = base_estudo.apply(elegibilidade,axis=1)

In [50]:
base_estudo.head()

,CUS_CUST_ID_SEL,TOTALGMV,DOMGMV,DOM,ENVGMV,status
0,"2,126.00",62.67,62.67,1,0,Não Opera nos Modais
1,"4,651.00","1,449.35",1449.35,1,0,Não Opera nos Modais
2,"5,824.00",276.99,276.99,1,0,Não Opera nos Modais
3,"14,667.00","1,553.02",995.06,2,0,Não Opera GMV Suficiente nos Dominios Chaves
4,"20,351.00","17,334.08",8096.77,3,8096.77,Não Opera GMV Suficiente nos Dominios Chaves


In [54]:
base_estudo['status'].value_counts()

Não Opera GMV Suficiente nos Dominios Chaves    72702
Não Opera nos Modais                            44800
Aprovado                                        25801
Name: status, dtype: int64

In [58]:
#Busca operação de GMV em 3 meses dos Sellers Aprovados

in_exp = base_estudo[base_estudo['status']=='Aprovado']['CUS_CUST_ID_SEL'].apply(str).unique()
in_exp = ','.join(in_exp)
q_gmv = """SELECT
  DATE_TRUNC(DIA_SERVER, MONTH), 
  CUS_CUST_ID_SEL,
  round(sum(F_TGMV_LC),2) as TOTALGMV,
 
FROM `meli-bi-data.EXPLOTACION.DATAMART_MKPLC` 

WHERE 1=1 
  and DIA_SERVER between '2022-03-01' and '2022-05-31'
  and SITE_ID = 'MLB'
  and cus_cust_id_sel in ({})
group by 1,2
""".format(in_exp)
res =  bq_client.query(q_gmv)
df_gmv = monta_df(res)


In [61]:
df_gmv = df_gmv.pivot_table(columns='f0_',index='CUS_CUST_ID_SEL',values='TOTALGMV')

In [64]:
df_gmv.reset_index(inplace=True)

In [66]:
df_gmv.head()

f0_,CUS_CUST_ID_SEL,2022-03-01,2022-04-01,2022-05-01
0,31958,"62,506.55","86,138.55","143,376.60"
1,42605,"3,843.00","2,210.06","2,888.50"
2,44643,"4,344.40","3,726.64","4,241.77"
3,50641,295.38,564.06,"3,897.31"
4,59502,"30,057.44","26,410.86","28,245.58"


In [69]:
final = base_estudo[base_estudo['status']=='Aprovado'].merge(df_gmv,how='left')

In [71]:
ws = sheet.worksheet('V0_Aprovados')
ws.clear()
gd.set_with_dataframe(worksheet=ws,dataframe=final,include_index=False,include_column_header=True,resize=True)

In [73]:
df.merge(base_estudo[['CUS_CUST_ID_SEL','status']])

,DOM2,CUS_CUST_ID_SEL,TOTALGMV,DOMGMV,ENVGMV,DOM%,DOM,status
0,LAUNDRY & DISHWASHERS,267892530,6016146.9,1691833.09,0,0.2812153888729013581766096835,LAUNDRY & DISHWASHERS,Não Opera GMV Suficiente nos Dominios Chaves
1,LAUNDRY & DISHWASHERS,185638169,6923246.17,728289.86,0,0.1051948525470386386679646262,LAUNDRY & DISHWASHERS,Não Opera GMV Suficiente nos Dominios Chaves
2,LAUNDRY & DISHWASHERS,234000251,8147649.51,204066.45,0,0.02504605159433275622087970743,LAUNDRY & DISHWASHERS,Não Opera GMV Suficiente nos Dominios Chaves
3,LAUNDRY & DISHWASHERS,323697852,285709.86,14954.62,0,0.05234198077728224010189917842,LAUNDRY & DISHWASHERS,Não Opera GMV Suficiente nos Dominios Chaves
4,LAUNDRY & DISHWASHERS,442421077,7682353.04,2061271.98,0,0.2683125820002669390471021623,LAUNDRY & DISHWASHERS,Não Opera GMV Suficiente nos Dominios Chaves
...,...,...,...,...,...,...,...,...
278,LAUNDRY & DISHWASHERS,588498521,151.51,151.51,0,1,LAUNDRY & DISHWASHERS,Não Opera nos Modais
279,LAUNDRY & DISHWASHERS,49851703,185.12,82.65,0,0.4464671564390665514261019879,LAUNDRY & DISHWASHERS,Não Opera GMV Suficiente nos Dominios Chaves
280,LAUNDRY & DISHWASHERS,1314402683,1805.19,1805.19,0,1,LAUNDRY & DISHWASHERS,Não Opera nos Modais
281,LAUNDRY & DISHWASHERS,1276073193,7498.21,657.3,0,0.08766092173998861061506679594,LAUNDRY & DISHWASHERS,Não Opera GMV Suficiente nos Dominios Chaves
